# 02 - 模型训练（新主线：delta + q_norm 横轴）

本 notebook 仅保留并演示当前项目的**唯一有效**输入定义与训练工作流。

## 输入定义（统一三通道）
- `v_delta`：电压形状（归一化后）相对前10循环均值的差值
- `i_delta`：电流形状（归一化后）相对前10循环均值的差值
- `q_norm`：归一化容量进度轴（0→1，作为横坐标；不做 baseline 扣除）

对应到代码：
- `src/data/feature.py:create_sequence` 返回 `(num_samples, 3)` 的 `delta_seq=[v_delta,i_delta,q_norm]`
- `src/data/feature.py:create_heatmap` 返回 `(window_size, num_samples, 3)` 的堆叠/插值热力图
- `src/data/battery_data.py:compute_soh` 使用前10循环容量均值作为 Q0：`SOH = Q/Q0`

## 训练入口
建议使用 `src/experiments.py` 的 `ExperimentConfig + run_experiments`，它会统一数据加载、建模、训练与评估。
本 notebook 仅做：
1) 构建 dataloader 并检查 batch 字段（包含 `v_delta/i_delta/t_delta(q_norm)`）
2) 用 `run_experiments` 跑一个最小例子（1-2 epoch）确认流程通。


In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import torch

from src.data import load_processed_batteries, ProtocolInvariantTransform
from src.data.dataset import create_dataloaders
from src.experiments import ExperimentConfig, run_experiments, print_results_table

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)


In [ ]:
# ========== 最小配置（你可按需修改） ==========
DATASET_NAME = 'MATR'
INPUT_TYPE = 'sequence'        # 'features' | 'sequence' | 'image'
MODEL_NAME = 'lstm'          # image 推荐: 'cnn2d' 或 'vit'
TARGET_TYPE = 'both'        # 'soh' | 'rul' | 'both'

NUM_SAMPLES = 200
WINDOW_SIZE = 100
BATCH_SIZE = 8

RANDOM_TRUNCATE = True
RANDOM_EOL_THRESHOLD = True

# 重要：现在三通道是 [v_delta, i_delta, q_norm]
# 归一化用 ProtocolInvariantTransform（按样本做 minmax/maxabs/0-1）
FEATURE_TRANSFORM = ProtocolInvariantTransform(channels=['v_delta', 'i_delta', 'q_norm'])


In [ ]:
batteries = load_processed_batteries(f'../data/processed/{DATASET_NAME}')
print('batteries:', len(batteries))

train_loader, val_loader, test_loader = create_dataloaders(
    batteries=batteries,
    input_type=INPUT_TYPE,
    target_type=TARGET_TYPE,
    num_samples=NUM_SAMPLES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    seed=42,
    random_truncate=RANDOM_TRUNCATE,
    random_eol_threshold=RANDOM_EOL_THRESHOLD,
    feature_transform=FEATURE_TRANSFORM,
)

batch = next(iter(train_loader))
print('feature:', tuple(batch['feature'].shape))
print('label:', tuple(batch['label'].shape))
print('soh:', tuple(batch['soh'].shape))
print('rul:', tuple(batch['rul'].shape), 'valid=', int((batch['rul_mask']>0).sum().item()))
print('v_delta:', tuple(batch['v_delta'].shape))
print('i_delta:', tuple(batch['i_delta'].shape))
print('t_delta(q_norm):', tuple(batch['t_delta'].shape))


## 用 experiments 跑一个最小训练（建议 1-2 epoch）
注意：这里的目的不是追求指标，而是确认新 pipeline + 模型可跑通。

In [ ]:
config = ExperimentConfig(
    datasets=[DATASET_NAME],
    input_type=INPUT_TYPE,
    models=[MODEL_NAME],
    target_type=TARGET_TYPE,
    num_samples=NUM_SAMPLES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    epochs=2,
    lr=1e-3,
    random_truncate=RANDOM_TRUNCATE,
    random_eol_threshold=RANDOM_EOL_THRESHOLD,
    seed=42,
)

results = run_experiments(config, processed_dir='../data/processed')
print_results_table(results)
